In [377]:
## Required Libraries
import re
import nltk
import string
import numpy as np
import pandas as pd
nltk.download('punkt')
nltk.download('twitter_samples')
from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('stopwords')
stemmer = PorterStemmer()
stopwords_english = stopwords.words('english')
punctuations = string.punctuation

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vinu1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\vinu1\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vinu1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [378]:
### Loading the Dataframe cell 

df_test = pd.read_csv('C:\\Users\\vinu1\\Personality_perception\\data\\Tweets\\BarackObama_tweets.csv')
# df.drop(['Unnamed: 0', 'Index',  
#                 'Location', 'Subcategory',
#        'Unnamed: 5'], axis=1, inplace=True)
# df['text'] = df['text'].str.lower()
# df['text']

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')
train_pos = all_positive_tweets[:]
train_neg = all_negative_tweets[:]
train_x = train_pos+train_neg
train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
df_train = pd.DataFrame()
df_train["text"] = train_x
df_train["sentiment"] = train_y

In [379]:
class CleanData:
    
    def __init__(self, df_test,df_train):
         self.df_test_cleaned = df_test
         self.df_train_cleaned = df_train
         self.Openness_list = []
         self.Agreeableness_list = []
         self.all_positive_tweets = []
         self.all_negative_tweets = []
         self.stopwords_english = stopwords.words('english')
        
    def read_persona_dict(self, Personality):
        for i in Personality:
            path = f"C:\\Users\\vinu1\\Personality_perception\\data\\Dictionary\\{i}.txt"
            with open(path) as file:
                if i == "Openness":
                   self.Openness_list = [line.rstrip() for line in file]
                   self.Openness_list = [re.sub(r"[^a-zA-Z0-9]", "", file) for file in self.Openness_list]
                elif i == "Agreeableness":
                   self.Agreeableness_list = [line.rstrip() for line in file]
                   self.Agreeableness_list = [re.sub(r"[^a-zA-Z0-9]", "", file) for file in self.Agreeableness_list]


        
    
    def pre_processing_dataframe(self,df):
#         df["text"] = df["text"].str.lower()
#         word_token_list = []
#         for i in range(len(df.index)):
#             word_token = word_tokenize(df["text"][i])
#             word_token_list.append(word_token)
#         df1 = pd.DataFrame(word_token_list)
#         df1['tokenized_tweets']=df1.apply(lambda row: row.dropna().tolist(), axis=1)
#         df_token = df1[['tokenized_tweets']]
#         df = pd.concat([df.reset_index(drop=True), df_token], axis=1)
        
        stopwords_english = stopwords.words('english')
        punctuations = string.punctuation
        stemmer = PorterStemmer()
        tokenizer = TweetTokenizer(preserve_case = False, strip_handles=True, reduce_len = True)
        #### Removing the unwanted things in the tweets
        df["text"]=df["text"].apply(lambda x: re.sub(r'RT[\s]+','', str(x)))
        df['text']=df['text'].apply(lambda x: re.sub(r"http\S+",'', str(x)))
        df['text']=df['text'].apply(lambda x: re.sub(r'#', '', str(x)))
        df['tokenized_tweets']=df['text'].apply(lambda x: word_tokenize(x))
        
        df['tokenized_tweets'] = df['tokenized_tweets'].apply(lambda x: [word.lower() for word in x if (word not in stopwords_english and word not in punctuations)])


        
    def pre_process(self, x):
        if x == "train":
            self.pre_processing_dataframe(self.df_train_cleaned)
        elif x == "test":
            self.pre_processing_dataframe(self.df_test_cleaned)
        
    def get_persona_scores(self):
        max_list = []
        persona_list = []
        open_count, agree_count = 0 , 0
        open_ratio, agree_ratio = 0 , 0

        for i in range(len(self.df_test_cleaned.index)):
            max_score = 0
            open_count, agree_count = 0 , 0
            open_ratio, agree_ratio = 0 , 0
            for j in range(len(self.df_test_cleaned["tokenized_tweets"][i])):

                if (self.df_test_cleaned["tokenized_tweets"][i][j] in self.Openness_list):
                    open_count += 1

                    open_ratio = open_count/len(self.Openness_list)
                if (self.df_test_cleaned["tokenized_tweets"][i][j] in self.Agreeableness_list):
                    agree_count +=1
                    agree_ratio = agree_count/len(self.Agreeableness_list)
            max_score = max(open_ratio,agree_ratio)
            max_list.append(max_score)
            

            if max_score == 0:
                x = "Unknown"
            elif max_score == open_ratio:
                x = "Openness"
            elif max_score == agree_ratio:
                x = "Agreeableness"

            persona_list.append(x)


        self.df_test_cleaned["personality_score"] = max_list
        self.df_test_cleaned["Personality"] = persona_list
    



In [380]:
class Training_predict_data(CleanData):
   def create_frequency(self):
        freq_d = {}
        for i in range(len(self.df_train_cleaned.index)):
            for word in self.df_train_cleaned["tokenized_tweets"][i]:
                pair = (word, self.df_train_cleaned["sentiment"][i])
                if pair in freq_d:
                    freq_d[pair] += 1
                else:
                    freq_d[pair] = freq_d.get(pair, 1)
    
        return freq_d
    
   def train_naive_bayes(self,freqs, train_x, train_y):
        loglikelihood = {}
        logprior = 0

        unique_words = set([pair[0] for pair in freqs.keys()])

        V = len(unique_words)

        N_pos = N_neg = 0
        for pair in freqs.keys():
            if pair[1]>0:
                N_pos+=freqs[(pair)]

            else:
                N_neg += freqs[(pair)]
        D = train_y.shape[0]

        D_pos = sum(train_y)

        D_neg = D-sum(train_y)

        logprior = np.log(D_pos) - np.log(D_neg)

        for word in unique_words:
            freq_pos = freqs.get((word, 1),0)
            freq_neg = freqs.get((word, 0), 0)
            p_w_pos = (freq_pos+1)/(N_pos+V)
            p_w_neg = (freq_neg+1)/(N_neg+V)
            loglikelihood[word] = np.log(p_w_pos/p_w_neg)
        return logprior, loglikelihood
    
   def naive_bayes_predict(self,logprior,loglikelihood):
       sentiment_score = []
       for i in range(len(self.df_test_cleaned.index)):
               p = 0
               p+=logprior
               for word in self.df_test_cleaned["tokenized_tweets"][i]:
                 if word in loglikelihood:
                   p+=loglikelihood[word]
               sentiment_score.append(p)
       self.df_test_cleaned["sentiment_score"] = sentiment_score
    
   def sentiment_persona(self):
      sentiment_persona = []
      for i in range(len(self.df_test_cleaned.index)):
                score  = self.df_test_cleaned["sentiment_score"][i]
                if (-0.5 < score < 0.5):
                    x  = "Neutral"
                elif (score < -0.5):
                    x = "Negative"
                elif (score > 0.5):
                    x = "Positive"
                sentiment_persona.append(x)
      self.df_test_cleaned["sentiment_persona"] = sentiment_persona

In [381]:
Personality = ['Openness', 'Agreeableness']
# clean_data = CleanData(df)
train_data = Training_predict_data(df_test, df_train)
train_data.read_persona_dict(Personality)
train_data.pre_process("test")
train_data.pre_process("train")
train_data.get_persona_scores()
freq_d=train_data.create_frequency()

In [382]:
logprior, loglikelihood = train_data.train_naive_bayes(freq_d, train_x, train_y)

train_data.naive_bayes_predict(logprior,loglikelihood)

In [383]:
train_data.sentiment_persona()
train_data.df_test_cleaned

,id,created_at,text,tokenized_tweets,personality_score,Personality,sentiment_score,sentiment_persona
0,1510286021182771200,2022-04-02 16:00:21+00:00,"I heard Betty Reid Soskin is retiring at 100, ...","[i, heard, betty, reid, soskin, retiring, 100,...",0.000000,Unknown,-0.658966,Negative
1,1510050583197237252,2022-04-02 00:24:49+00:00,"I'm proud of Titus, Ashton, Thaddeus, Jalen, M...","[i, 'm, proud, titus, ashton, thaddeus, jalen,...",0.000000,Unknown,-1.700571,Negative
2,1507085038940262408,2022-03-24 20:00:48+00:00,Today @POTUS announced that the U.S. will be w...,"[today, potus, announced, u.s., welcoming, 100...",0.016393,Agreeableness,0.630128,Positive
3,1506730193712467970,2022-03-23 20:30:46+00:00,As the first woman to serve as America’s top d...,"[as, first, woman, serve, america, ’, top, dip...",0.013699,Openness,-0.451976,Neutral
4,1506718741459091464,2022-03-23 19:45:16+00:00,"The other day, I called up DonnaMarie, Steve, ...","[the, day, i, called, donnamarie, steve, amy, ...",0.000000,Unknown,2.644516,Positive
...,...,...,...,...,...,...,...,...
3245,576151284757073920,2015-03-12 22:42:25+00:00,@JimmyKimmelLive: TBT Just two dudes sharing a...,"[jimmykimmellive, tbt, just, two, dudes, shari...",0.000000,Unknown,1.197737,Positive
3246,576133874511519744,2015-03-12 21:33:14+00:00,"We've already tripled wind energy in America, ...","[we, 've, already, tripled, wind, energy, amer...",0.013699,Openness,-1.763139,Negative
3247,576104761679708160,2015-03-12 19:37:33+00:00,Who will take home the title of ChampionDenier...,"[who, take, home, title, championdenier, vote]",0.000000,Unknown,1.105253,Positive
3248,575764044138708993,2015-03-11 21:03:39+00:00,@FLOTUS: If you submit a video on your high sc...,"[flotus, if, submit, video, high, school, fafs...",0.016393,Agreeableness,1.467033,Positive
